In [1]:
!pip install evaluate sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 868.5 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 1.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 5.6 MB/s eta 0:00:00


In [2]:
!pip uninstall -y transformers accelerate evaluate rouge_score
!pip install transformers accelerate evaluate rouge_score

Found existing installation: transformers 4.44.0
Uninstalling transformers-4.44.0:
  Successfully uninstalled transformers-4.44.0
Found existing installation: accelerate 0.33.0
Uninstalling accelerate-0.33.0:
  Successfully uninstalled accelerate-0.33.0
Found existing installation: evaluate 0.4.3
Uninstalling evaluate-0.4.3:
  Successfully uninstalled evaluate-0.4.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 635.6 kB/s eta 0:00:00 0:00:01
  Using cached evaluate-0.4.3-py3-none-any.whl.metadata (9.2 kB)
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 28.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 20.1 MB/s eta 0:00:00
Using cached evaluate-0.4.3-py3-none-any.whl (84 kB)
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=3d4ef617846a0c3eaab3becc48077072285d81de5c23a7934907111e69059751
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89

In [3]:
!pip install --upgrade tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 6.8 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.16.2
    Uninstalling tensorboard-2.16.2:
      Successfully uninstalled tensorboard-2.16.2
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.16.1
    Uninstalling tensorflow-2.16.1:
      Successfully uninstalled tensorflow-2.16.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.9.1 requires tensorflow~=2.16.1, but you have tensorflow 2.17.0 which is incompatible.
tensorflow-text 2.16.1 requires tensorflow<2.17,>=2.16.1; platform_machine != "arm64" or platform_system != "Darwin", but you have tensorflow 2.17.0 which is incompatible.
tf-keras 2.1

In [4]:
import os
import pandas as pd
import re
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling
from datasets import Dataset, DatasetDict
import evaluate
import torch

2024-09-15 06:36:52.050221: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-15 06:36:52.072357: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-15 06:36:52.080561: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
# Ensure you are using the correct device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [6]:
import evaluate
sacrebleu_metric = evaluate.load("sacrebleu")
rouge_metric = evaluate.load('rouge')

In [46]:
import re
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)  # Remove special characters
    return text

In [8]:
# Function to move tensors to the correct device
def move_to_device(batch, device):
    batch = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in batch.items()}
    return batch

In [9]:
def convert_csv_to_txt(csv_path, output_path, lowercase=False, clean=False):
    with open(csv_path, 'r', encoding='windows-1252') as csv_file, open(output_path, 'w', encoding='utf-8') as txt_file:
        csv_reader = csv.reader(csv_file)
        header = next(csv_reader)

        # Extracting the question and answer based on their header titles
        question_idx = 0
        answer_idx = 1
        
        for row in csv_reader:
            question = row[question_idx].strip() if row[question_idx] else ""
            answer = row[answer_idx].strip() if row[answer_idx] else ""

            if lowercase:
                question = question.lower()
                answer = answer.lower()
            if clean:
                question = clean_text(question)
                answer = clean_text(answer)

            if question and answer:
                txt_file.write(f"Question: {question}\nAnswer: {answer}\n\n")

In [10]:
dataset_path ='/kaggle/input/qa-dataset'

In [11]:
import csv

# Path to your CSV file
csv_path = dataset_path + '/train_1.csv'

# Open the CSV file and print the header
with open(csv_path, 'r', encoding='windows-1252') as csv_file:
    csv_reader = csv.reader(csv_file)
    header = next(csv_reader)
    print("CSV Header:", header)


CSV Header: ['Question :What is the role of Siamese networks in domain adaptation?', 'Answer :Siamese networks can be used to align the representations of source and target domains by minimizing the discrepancy between them, making the model more robust to domain shifts.']


In [12]:
# Convert CSV files to text format 
convert_csv_to_txt(dataset_path + '/train_1.csv', 'train.txt', lowercase=True, clean=True)
convert_csv_to_txt(dataset_path + '/dev_1.csv', 'dev.txt', lowercase=True, clean=True)

In [13]:
convert_csv_to_txt(dataset_path + '/test_1.csv', 'test.txt', lowercase=True, clean=True)

In [14]:
train_txt_file = '/kaggle/working/train.txt'
dev_txt_file = '/kaggle/working/dev.txt'
test_txt_file = '/kaggle/working/test.txt'

In [15]:
def load_txt_data(file_path):
    questions = []
    answers = []
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read().split("\n\n")  
        for entry in content:
            if "Question:" in entry and "Answer:" in entry:
                question = re.search(r"Question:(.+)", entry)
                answer = re.search(r"Answer:(.+)", entry)
                if question and answer:
                    questions.append(question.group(1).strip())
                    answers.append(answer.group(1).strip())
    return questions, answers

In [16]:
# Load datasets
train_questions, train_answers = load_txt_data('/kaggle/working/train.txt')
val_questions, val_answers = load_txt_data('/kaggle/working/dev.txt')
test_questions, test_answers = load_txt_data('/kaggle/working/test.txt')

# Convert to a format compatible with HuggingFace Dataset
train_data = {
    'question': train_questions,
    'answer': train_answers
}

val_data = {
    'question': val_questions,
    'answer': val_answers
}

test_data = {
    'question': test_questions,
    'answer': test_answers
}

In [17]:
print("Number of train questions:", len(train_questions))
print("Number of train answers:", len(train_answers))
print("Number of val questions:", len(val_questions))
print("Number of val answers:", len(val_answers))
print("Number of val questions:", len(test_questions))
print("Number of val answers:", len(test_answers))

Number of train questions: 1984
Number of train answers: 1984
Number of val questions: 247
Number of val answers: 247
Number of val questions: 248
Number of val answers: 248


In [18]:
train_df = pd.DataFrame(train_data)
val_df = pd.DataFrame(val_data)
test_df = pd.DataFrame(test_data)

In [19]:
print(train_df.shape)
print(test_df.shape)
print(val_df.shape)

(1984, 2)
(248, 2)
(247, 2)


In [20]:
# Convert DataFrames to Dataset
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

datasets = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset
})

In [21]:
len(train_dataset)

1984

In [22]:
# Load pre-trained tokenizer and model
model_name='gpt2'
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained('gpt2')

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [23]:
def preprocess_function(data):
    # Add a separator token between question and answer for clearer distinction
    inputs = [f"Question: {q} [SEP] Answer: {a}" for q, a in zip(data['question'], data['answer'])]

    # Dynamically adjust padding for efficiency in training
    model_inputs = tokenizer(
        inputs,
        max_length=256,              # Truncate to 256 tokens max
        truncation=True,             # Truncate long sequences
        padding='max_length',        # Pad to max length or dynamically for batch padding
        return_tensors="pt"          # Return PyTorch tensors
    )

    # Create labels for the model, masking the padding tokens (-100)
    labels = model_inputs['input_ids'].clone()
    labels[labels == tokenizer.pad_token_id] = -100
    model_inputs['labels'] = labels

    return model_inputs

In [24]:
tokenized_train_dataset = datasets['train'].map(preprocess_function, batched=True)
tokenized_val_dataset = datasets['validation'].map(preprocess_function, batched=True)


# Data Collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

Map:   0%|          | 0/1984 [00:00<?, ? examples/s]

Map:   0%|          | 0/247 [00:00<?, ? examples/s]

In [25]:
# Training Arguments
training_args = TrainingArguments(
       evaluation_strategy="epoch",
    output_dir='./results',
    overwrite_output_dir=True,
    learning_rate=3e-5,                 # Lower learning rate for more stable training
    per_device_train_batch_size=4,      # Tune based on your GPU memory
    per_device_eval_batch_size=4,
    num_train_epochs=10,               
    save_strategy="epoch",
    save_total_limit=1,
    logging_dir='./logs',
    logging_steps=100,
    warmup_steps=200,                   
    weight_decay=0.01,                  
    gradient_accumulation_steps=2
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=None
)

# Train the model
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,4.181600,2.853980
2,2.944100,2.544305
3,2.651400,2.428147
4,2.550700,2.367625
5,2.388000,2.327686
6,2.341500,2.300028
7,2.317200,2.283758
8,2.276500,2.272247
9,2.231900,2.265594
10,2.239000,2.263248


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

TrainOutput(global_step=1240, training_loss=2.55921747761388, metrics={'train_runtime': 1124.5758, 'train_samples_per_second': 17.642, 'train_steps_per_second': 1.103, 'total_flos': 2592016957440000.0, 'train_loss': 2.55921747761388, 'epoch': 10.0})

In [26]:
trainer.save_model("./results/final_model")

In [27]:
!zip -r /kaggle/working/results/final_model.zip /kaggle/working/results/final_model

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


  adding: kaggle/working/results/final_model/ (stored 0%)
  adding: kaggle/working/results/final_model/model.safetensors (deflated 7%)
  adding: kaggle/working/results/final_model/special_tokens_map.json (deflated 74%)
  adding: kaggle/working/results/final_model/training_args.bin (deflated 51%)
  adding: kaggle/working/results/final_model/generation_config.json (deflated 24%)
  adding: kaggle/working/results/final_model/merges.txt (deflated 53%)
  adding: kaggle/working/results/final_model/config.json (deflated 52%)
  adding: kaggle/working/results/final_model/tokenizer_config.json (deflated 55%)
  adding: kaggle/working/results/final_model/vocab.json (deflated 68%)


In [28]:
from safetensors.torch import load_file as load_safetensors
model_name='gpt2'
model_path = "/kaggle/working/results/final_model"
state_dict = load_safetensors(f"{model_path}/model.safetensors")
model = GPT2LMHeadModel.from_pretrained(model_path, state_dict=state_dict)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [29]:
rouge = evaluate.load("rouge")

In [30]:
# Set pad_token to eos_token if it's not already set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

In [31]:
def generate_text(sequence, max_new_tokens=50):
    # Encode the input sequence and move it to the correct device
    inputs = tokenizer(sequence, return_tensors='pt', padding=True, truncation=True)
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs.get('attention_mask', None).to(device)  # Get attention mask if available

    # Ensure the model is on the same device
    model.to(device)

    # Generate text with the model
    try:
        output = model.generate(
            input_ids,
            attention_mask=attention_mask,
            do_sample=True,
            max_length=input_ids.size(1) + max_new_tokens,
            pad_token_id=tokenizer.pad_token_id,  # Ensure this is set correctly
            temperature=1.0,  # Control randomness
            top_k=50,        # Control diversity
            top_p=0.95       # Control diversity
        )
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        return generated_text
    except Exception as e:
        print(f"Error during text generation: {e}")
        return ""

In [36]:
def question_answer(question, max_new_tokens=50):
    # Ensure the model is in evaluation mode and on the correct device
    model.eval()
    model.to(device)  # Move model to the correct device (GPU or CPU)
    
    # Prepare the input prompt
    prompt = f"Question: {question} Answer:"
    
    # Tokenize the prompt and ensure the tensors are on the same device as the model
    inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to(device)
    
    try:
        # Generate the output from the model
        output = model.generate(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_length=inputs['input_ids'].size(1) + max_new_tokens,
            pad_token_id=tokenizer.pad_token_id,
            temperature=1.0,
            top_k=50,
            top_p=0.95,
            do_sample=True  # Enable sampling for top_k and top_p to work
        )

        # Decode the generated output
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        
        # Extract only the answer from the generated text
        generated_answer = generated_text.replace(f"Question: {question} Answer:", "").strip()
        return generated_answer
    except Exception as e:
        print(f"Error during text generation: {e}")
        return ""

print(question_answer("What is feature engineering for unstructured data?"))

feature engineering for unstructured data involves finding and refining techniques and techniques that improve human performance in complex tasks requiring detailed metrics or data access without compromising human skills in general knowledge or ability to process data efficiently or perform tasks complex enough for humans to capture and


In [37]:
question_answer('What is the mean absolute error (MAE)?')

'answer the mean absolute error describes the probability of the predicted results with respect to both the sample and its nearest neighbor the standard deviation or variance of the prediction in a given sample it is a measure of how well predicted the predicted results are expressed in the data'

In [38]:
print(question_answer('What is Q-Learning and how does it work?'))

Qlearning uses a series of computer algorithms to learn the structure and context of data and models used in scientific data analysis projects where the models come into play as the data becomes more efficient. Qlearning models the human brain during natural language processing in order to


In [39]:
question_answer('What is the difference between BERT and RoBERTa?')

'answer Bert is the simplest model for extracting data from text using a binary representation while robert is simpler and simpler for extracting data from a regularization process using either bert or robert but different functions for text generation and analysis in both cases are'

In [40]:
question_answer("What is the primary function of a Generative Adversarial Network (GAN)?")

'answer a primary function is the ability of a network to process information from a sequence to produce a unique and meaningful output by applying a high degree of precision to the sequence that can be used for semantic modeling or to learn the relationships between words and actions through'

In [44]:
question_answer("What Optimization Algorithms are available in Scikit-Learn?")

'Optimization algorithms for deep learning are common among other algorithms including nlp and rote learning which are available under the Scikit umbrella and available under different technologies including neural network gated computing tools and reinforcement learning tools as well as through other neural networks ('

In [48]:
# Ensure the clean_answer function is defined
def clean_answer(answer):
    return answer.strip().lower()

# Generate predictions for the test set
predicted_answers = [question_answer(q, max_new_tokens=50) for q in test_questions]

# Clean test answers
test_answers_cleaned = [clean_answer(a) for a in test_answers]

# Check if the lengths match
if len(predicted_answers) == len(test_answers_cleaned):
    # Compute ROUGE scores
    rouge_results = rouge.compute(predictions=predicted_answers, references=test_answers_cleaned)
    
    # Display ROUGE Scores
    print("ROUGE Scores:")
    for rouge_type, score in rouge_results.items():
        print(f"{rouge_type}: {score}")
else:
    print("Error: The number of predicted answers and test answers do not match.")

ROUGE Scores:
rouge1: 0.2385221287321419
rouge2: 0.06923877974548409
rougeL: 0.19462264064174561
rougeLsum: 0.1944902155357165


[Download File](sandbox:/kaggle/working/results/final_model.zip)